In [1]:
from utils.refactor_class import *

In [2]:
import pandas, numpy, sklearn

# Demo

In [3]:
objlib = sklearn

In [4]:
print("Obtaining inheritance_info", objlib.__name__, objlib.__version__)

inheritance_info = get_all_classes_in_package(objlib)
inheritance_info = sorted(list(inheritance_info), key=lambda x: str(f"{x[0].__module__}.{x[0].__qualname__}"))

for cls, base_classes in inheritance_info:
    print(f"Class: {cls.__module__}.{cls.__name__}")
    if base_classes:
        print(f"  Inherits from {len(base_classes)} classes:")
        for base in base_classes:
            print(f"    {base.__module__}.{base.__name__}")
    else:
        print("  No base class.")

Obtaining inheritance_info sklearn 1.5.2
1.5.2


  warnings.warn(


Class: sklearn._loss.link.BaseLink
  No base class.
Class: sklearn._loss.link.HalfLogitLink
  Inherits from 1 classes:
    sklearn._loss.link.BaseLink
Class: sklearn._loss.link.IdentityLink
  Inherits from 1 classes:
    sklearn._loss.link.BaseLink
Class: sklearn._loss.link.Interval
  No base class.
Class: sklearn._loss.link.LogLink
  Inherits from 1 classes:
    sklearn._loss.link.BaseLink
Class: sklearn._loss.link.LogitLink
  Inherits from 1 classes:
    sklearn._loss.link.BaseLink
Class: sklearn._loss.link.MultinomialLogit
  Inherits from 1 classes:
    sklearn._loss.link.BaseLink
Class: sklearn._loss.loss.AbsoluteError
  Inherits from 1 classes:
    sklearn._loss.loss.BaseLoss
Class: sklearn._loss.loss.BaseLoss
  No base class.
Class: sklearn._loss.loss.ExponentialLoss
  Inherits from 1 classes:
    sklearn._loss.loss.BaseLoss
Class: sklearn._loss.loss.HalfBinomialLoss
  Inherits from 1 classes:
    sklearn._loss.loss.BaseLoss
Class: sklearn._loss.loss.HalfGammaLoss
  Inherits from

In [5]:
cls2defmethods, cls2outmethods, defmethods2cls = get_classes_defmethod_mappings(inheritance_info, verbose=False)
for cls, defmethods in cls2defmethods.items():
    print(f"{cls.__qualname__} @ {cls.__module__} : ({len(defmethods)})")
    print('\n'.join(['\t'+x.__qualname__ for x in defmethods]))

BaseLink @ sklearn._loss.link : (2)
	BaseLink.link
	BaseLink.inverse
HalfLogitLink @ sklearn._loss.link : (2)
	HalfLogitLink.link
	HalfLogitLink.inverse
IdentityLink @ sklearn._loss.link : (1)
	IdentityLink.link
Interval @ sklearn._loss.link : (2)
	Interval.__post_init__
	Interval.includes
LogLink @ sklearn._loss.link : (2)
	LogLink.link
	LogLink.inverse
LogitLink @ sklearn._loss.link : (2)
	LogitLink.link
	LogitLink.inverse
MultinomialLogit @ sklearn._loss.link : (3)
	MultinomialLogit.symmetrize_raw_prediction
	MultinomialLogit.link
	MultinomialLogit.inverse
AbsoluteError @ sklearn._loss.loss : (2)
	AbsoluteError.__init__
	AbsoluteError.fit_intercept_only
BaseLoss @ sklearn._loss.loss : (11)
	BaseLoss.fit_intercept_only
	BaseLoss.in_y_true_range
	BaseLoss.loss_gradient
	BaseLoss.gradient
	BaseLoss.constant_to_optimal_zero
	BaseLoss.in_y_pred_range
	BaseLoss.gradient_hessian
	BaseLoss.init_gradient_and_hessian
	BaseLoss.loss
	BaseLoss.__call__
	BaseLoss.__init__
ExponentialLoss @ sklea

In [6]:
inheritance_info_eligible = [(cls, _) for cls, _ in inheritance_info if get_class_source_info(cls)[0] is not None]
inheritance_info_eligible_classes_dict = {cls: base_classes for cls, base_classes in inheritance_info_eligible}

inheritance_info_eligible_384 = []
for cls, _ in inheritance_info_eligible:
    contentvals = [v[0] for v in get_cls_content_list(cls, cls2defmethods, skipwhitespace=False)]
    if (len(norm_code(inspect.getsource(cls)).splitlines()) >= 3 and len(inspect.getsource(cls))<=2048):
        inheritance_info_eligible_384.append( (cls, _) )
print(len(inheritance_info), "=>", len(inheritance_info_eligible), "=>", len(inheritance_info_eligible_384), end = ' ')

709 => 552 => 181 

In [7]:
for idx, (cls, _) in enumerate(inheritance_info):
    class_file, class_start_line, class_end_line = get_class_source_info(cls)
    if class_start_line == None:
        print(f"Skipped checking on {cls.__qualname__}.")
        continue

    clscode = inspect.getsource(cls)
    
    contents = get_cls_content_list(cls, cls2defmethods, skipwhitespace=False)
    contentvals = [v[0] for v in contents]
    valist, succ = ClassNoRefactor().refactor(contentvals, cls)
    regencode = contentvals2str(valist)
    assert succ and regencode == clscode
    valist, succ = ClassMethodShuffle().refactor(contentvals, cls)
    shuffledcode = contentvals2str(valist)
    assert succ == False or (shuffledcode != clscode and len(shuffledcode) == len(clscode))
    valist, succ = ClassAppendInheritedMethods().refactor(contentvals, cls, inheritance_info_eligible_classes_dict, cls2outmethods, defmethods2cls)
    appendedcode = contentvals2str(valist)
    assert succ == False or (len(appendedcode) > len(clscode) and appendedcode.startswith(clscode))

    print(f"Finished checking on {cls.__module__}.{cls.__qualname__}.")

Finished checking on sklearn._loss.link.BaseLink.
Finished checking on sklearn._loss.link.HalfLogitLink.
Finished checking on sklearn._loss.link.IdentityLink.
Finished checking on sklearn._loss.link.Interval.
Finished checking on sklearn._loss.link.LogLink.
Finished checking on sklearn._loss.link.LogitLink.
Finished checking on sklearn._loss.link.MultinomialLogit.
Finished checking on sklearn._loss.loss.AbsoluteError.
Finished checking on sklearn._loss.loss.BaseLoss.
Finished checking on sklearn._loss.loss.ExponentialLoss.
Finished checking on sklearn._loss.loss.HalfBinomialLoss.
Finished checking on sklearn._loss.loss.HalfGammaLoss.
Finished checking on sklearn._loss.loss.HalfMultinomialLoss.
Finished checking on sklearn._loss.loss.HalfPoissonLoss.
Finished checking on sklearn._loss.loss.HalfSquaredError.
Finished checking on sklearn._loss.loss.HalfTweedieLoss.
Finished checking on sklearn._loss.loss.HalfTweedieLossIdentity.
Finished checking on sklearn._loss.loss.HuberLoss.
Finished 

Finished checking on sklearn.ensemble._stacking.StackingClassifier.
Finished checking on sklearn.ensemble._stacking.StackingRegressor.
Finished checking on sklearn.ensemble._stacking._BaseStacking.
Finished checking on sklearn.ensemble._voting.VotingClassifier.
Finished checking on sklearn.ensemble._voting.VotingRegressor.
Finished checking on sklearn.ensemble._voting._BaseVoting.
Finished checking on sklearn.ensemble._weight_boosting.AdaBoostClassifier.
Finished checking on sklearn.ensemble._weight_boosting.AdaBoostRegressor.
Finished checking on sklearn.ensemble._weight_boosting.BaseWeightBoosting.
Finished checking on sklearn.exceptions.ConvergenceWarning.
Finished checking on sklearn.exceptions.DataConversionWarning.
Finished checking on sklearn.exceptions.DataDimensionalityWarning.
Finished checking on sklearn.exceptions.EfficiencyWarning.
Finished checking on sklearn.exceptions.FitFailedWarning.
Finished checking on sklearn.exceptions.InconsistentVersionWarning.
Finished checking

Finished checking on sklearn.linear_model._coordinate_descent.MultiTaskLassoCV.
Finished checking on sklearn.linear_model._glm._newton_solver.NewtonCholeskySolver.
Finished checking on sklearn.linear_model._glm._newton_solver.NewtonSolver.
Finished checking on sklearn.linear_model._glm.glm.GammaRegressor.
Finished checking on sklearn.linear_model._glm.glm.PoissonRegressor.
Finished checking on sklearn.linear_model._glm.glm.TweedieRegressor.
Finished checking on sklearn.linear_model._glm.glm._GeneralizedLinearRegressor.
Finished checking on sklearn.linear_model._huber.HuberRegressor.
Finished checking on sklearn.linear_model._least_angle.Lars.
Finished checking on sklearn.linear_model._least_angle.LarsCV.
Finished checking on sklearn.linear_model._least_angle.LassoLars.
Finished checking on sklearn.linear_model._least_angle.LassoLarsCV.
Finished checking on sklearn.linear_model._least_angle.LassoLarsIC.
Finished checking on sklearn.linear_model._linear_loss.LinearModelLoss.
Finished che

Finished checking on sklearn.model_selection._plot.ValidationCurveDisplay.
Finished checking on sklearn.model_selection._plot._BaseCurveDisplay.
Finished checking on sklearn.model_selection._search.BaseSearchCV.
Finished checking on sklearn.model_selection._search.GridSearchCV.
Finished checking on sklearn.model_selection._search.ParameterGrid.
Finished checking on sklearn.model_selection._search.ParameterSampler.
Finished checking on sklearn.model_selection._search.RandomizedSearchCV.
Finished checking on sklearn.model_selection._search_successive_halving.BaseSuccessiveHalving.
Finished checking on sklearn.model_selection._search_successive_halving.HalvingGridSearchCV.
Finished checking on sklearn.model_selection._search_successive_halving.HalvingRandomSearchCV.
Finished checking on sklearn.model_selection._search_successive_halving._SubsampleMetaSplitter.
Finished checking on sklearn.model_selection._split.BaseCrossValidator.
Finished checking on sklearn.model_selection._split.BaseSh

Finished checking on sklearn.tests.metadata_routing_common.MetaTransformer.
Finished checking on sklearn.tests.metadata_routing_common.NonConsumingClassifier.
Finished checking on sklearn.tests.metadata_routing_common.NonConsumingRegressor.
Finished checking on sklearn.tests.metadata_routing_common.WeightedMetaClassifier.
Finished checking on sklearn.tests.metadata_routing_common.WeightedMetaRegressor.
Finished checking on sklearn.tests.metadata_routing_common._Registry.
Finished checking on sklearn.tree._classes.BaseDecisionTree.
Finished checking on sklearn.tree._classes.DecisionTreeClassifier.
Finished checking on sklearn.tree._classes.DecisionTreeRegressor.
Finished checking on sklearn.tree._classes.ExtraTreeClassifier.
Finished checking on sklearn.tree._classes.ExtraTreeRegressor.
Skipped checking on ClassificationCriterion.
Skipped checking on Criterion.
Skipped checking on Entropy.
Skipped checking on FriedmanMSE.
Skipped checking on Gini.
Skipped checking on MAE.
Skipped checki

Finished checking on sklearn.utils.tests.test_estimator_checks.NoSparseClassifier.
Finished checking on sklearn.utils.tests.test_estimator_checks.NotInvariantPredict.
Finished checking on sklearn.utils.tests.test_estimator_checks.NotInvariantSampleOrder.
Finished checking on sklearn.utils.tests.test_estimator_checks.OneClassSampleErrorClassifier.
Finished checking on sklearn.utils.tests.test_estimator_checks.PartialFitChecksName.
Finished checking on sklearn.utils.tests.test_estimator_checks.PoorScoreLogisticRegression.
Finished checking on sklearn.utils.tests.test_estimator_checks.RaisesErrorInSetParams.
Finished checking on sklearn.utils.tests.test_estimator_checks.RequiresPositiveXRegressor.
Finished checking on sklearn.utils.tests.test_estimator_checks.RequiresPositiveYRegressor.
Finished checking on sklearn.utils.tests.test_estimator_checks.SetsWrongAttribute.
Finished checking on sklearn.utils.tests.test_estimator_checks.SparseTransformer.
Finished checking on sklearn.utils.tests

In [8]:
cls = sklearn.externals._arff.LODGeneratorData

clscontents = get_cls_content_list(cls, cls2defmethods, skipwhitespace=False)
clscontentvals = [v[0] for v in clscontents]
print(contentvals2str(clscontentvals), sep='\n')

class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(row) + 

In [9]:
# Class-level Inhr

vallist, succs = clscontentvals, []
vallist, succ = ClassAppendInheritedMethods().refactor(vallist, cls, inheritance_info_eligible_classes_dict, cls2outmethods, defmethods2cls)
print(succ, contentvals2str(clscontentvals), sep='\n')

False
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(r

In [10]:
# Class-level Shuf

vallist, succs = clscontentvals, []
vallist, succ = ClassMethodShuffle().refactor(vallist, cls)
print(succ, contentvals2str(clscontentvals), sep='\n')

True
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(ro

In [11]:
# Method-level IFF

vallist, succs = clscontentvals, []
vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.IfBranchFliper()) )
print(succ, contentvals2str(clscontentvals), sep='\n')

True
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(ro

In [12]:
# Method-level Loop

vallist, succs = clscontentvals, []
vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.While2For(), func_refacotr.For2While()) )
print(succ, contentvals2str(clscontentvals), sep='\n')

True
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(ro

In [13]:
# Method-level Iter

vallist, succs = clscontentvals, []
vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.List2Range()) )
print(succ, contentvals2str(clscontentvals), sep='\n')

True
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(ro

In [14]:
# Method-level Comm

vallist, succs = clscontentvals, []
vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.CommLaw()) )
print(succ, contentvals2str(clscontentvals), sep='\n')

True
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(ro

In [15]:
# Method-level Deco

vallist, succs = clscontentvals, []
vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.FnDecorator('@timeing', '@measure_memory_usage')) )
print(succ, contentvals2str(clscontentvals), sep='\n')

True
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(ro

In [16]:
# Method-level Param

vallist, succs = clscontentvals, []
vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.FnVarargAppender(), func_refacotr.FnKwargAppender()) )
print(succ, contentvals2str(clscontentvals), sep='\n')

True
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
                    'Instance %d has %d attributes, expected %d' %
                    (current_row, max(ro

In [ ]:
# Method-level Renm

vallist, succs = clscontentvals, []
vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.VarRenamer()) )
print(succ, contentvals2str(clscontentvals), sep='\n')

In [17]:
# Method-level Styl

vallist, succs = clscontentvals, []
vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.CamelSnakeExchange()) )
print(succ, contentvals2str(clscontentvals), sep='\n')

renaming current_row => currentRow
renaming num_attributes => numAttributes
renaming new_data => newData
True
class LODGeneratorData:
    def decode_rows(self, stream, conversors):
        for row in stream:
            values = _parse_values(row)

            if not isinstance(values, dict):
                raise BadLayout()
            try:
                yield {key: None if value is None else conversors[key](value)
                       for key, value in values.items()}
            except ValueError as exc:
                if 'float: ' in str(exc):
                    raise BadNumericalValue()
                raise
            except IndexError:
                # conversor out of range
                raise BadDataFormat(row)

    def encode_data(self, data, attributes):
        current_row = 0

        num_attributes = len(attributes)
        for row in data:
            new_data = []

            if len(row) > 0 and max(row) >= num_attributes:
                raise BadObject(
  

# Reproduction Script

In [ ]:
import os, datetime, pickle
import json

def dumpq(cls, newcontentvals, refactored, do_norm=True):
    return json.dumps({
        'repo': f"{objlib.__name__}",
        'url': f"{objlib.__name__}=={objlib.__version__}", 
        'last_update_at': "", 
        'file': inspect.getfile(cls)[70:], 
        'fn_id': "", 
        'content': contentvals2str(newcontentvals, do_norm=do_norm), 
        'class_fn': True, 
        'question_id': f"{objlib.__name__}/{cls.__module__}/{cls.__qualname__}", 
        'category': "coding", 
        'instruct': "Please complete the following code and only return the code.", 
        'turns': [], 
        'refactored': refactored
    })

def work(objlib, oprs, do_check=True, timetag='new', do_norm=True):
    if not os.path.exists(f"data_{timetag}/"):
        os.makedirs(f"data_{timetag}/")
    
    print("Obtaining inheritance_info", objlib.__name__, objlib.__version__, timetag)

    inheritance_info = get_all_classes_in_package(objlib)
    inheritance_info = sorted(list(inheritance_info), key=lambda x: str(f"{x[0].__module__}.{x[0].__qualname__}"))
    
    for cls, base_classes in inheritance_info:
        print(f"Class: {cls.__module__}.{cls.__name__}")
        if base_classes:
            print(f"  Inherits from {len(base_classes)} classes:")
            for base in base_classes:
                print(f"    {base.__module__}.{base.__name__}")
        else:
            print("  No base class.")
    print("@" * 50)

    cls2defmethods, cls2outmethods, defmethods2cls = get_classes_defmethod_mappings(inheritance_info, verbose=False)

    for cls, defmethods in cls2defmethods.items():
        print(f"{cls.__qualname__} @ {cls.__module__} : ({len(defmethods)})")
        print('\n'.join(['\t'+x.__qualname__ for x in defmethods]))

    print("@" * 50)

    inheritance_info_eligible = [(cls, _) for cls, _ in inheritance_info if get_class_source_info(cls)[0] is not None]
    global inheritance_info_eligible_classes_dict
    inheritance_info_eligible_classes_dict = {cls: base_classes for cls, base_classes in inheritance_info_eligible}

    inheritance_info_eligible_384 = []
    for cls, _ in inheritance_info_eligible:
        contentvals = [v[0] for v in get_cls_content_list(cls, cls2defmethods, skipwhitespace=False)]
        if (len(norm_code(inspect.getsource(cls)).splitlines()) >= 3 and len(inspect.getsource(cls))<=2048):
            inheritance_info_eligible_384.append( (cls, _) )
    print(len(inheritance_info), "=>", len(inheritance_info_eligible), "=>", len(inheritance_info_eligible_384), end = ' ')
    inheritance_info_eligible_384 = inheritance_info_eligible_384[:384]
    print("=>", len(inheritance_info_eligible_384))
    print("@" * 50)
    
    if do_check:
        for idx, (cls, _) in enumerate(inheritance_info):
            class_file, class_start_line, class_end_line = get_class_source_info(cls)
            if class_start_line == None:
                print(f"Skipped checking on {cls.__qualname__}.")
                continue

            clscode = inspect.getsource(cls)
            
            contents = get_cls_content_list(cls, cls2defmethods, skipwhitespace=False)
            contentvals = [v[0] for v in contents]
            valist, succ = ClassNoRefactor().refactor(contentvals, cls)
            regencode = contentvals2str(valist)
            assert succ and regencode == clscode
            valist, succ = ClassMethodShuffle().refactor(contentvals, cls)
            shuffledcode = contentvals2str(valist)
            assert succ == False or (shuffledcode != clscode and len(shuffledcode) == len(clscode))
            valist, succ = ClassAppendInheritedMethods().refactor(contentvals, cls, inheritance_info_eligible_classes_dict, cls2outmethods, defmethods2cls)
            appendedcode = contentvals2str(valist)
            assert succ == False or (len(appendedcode) > len(clscode) and appendedcode.startswith(clscode))

            print(f"Finished checking on {cls.__qualname__}.")
    
    norefactors, methodshuffles, appendinhmethodss, appendashuffle = [], [], [], []
    funcifs, funcloops, funcparams, funcvarrenamers, funcfndecorators = [], [], [], [], []
    funccamelsnakeexchange, funclist2range, funccommlaw = [], [], []
    allw = []
    
    oprs_set = frozenset(oprs)
    
    for idx, (cls, _) in enumerate(inheritance_info_eligible_384):
        print(f"Start working on {cls.__qualname__} @ {cls.__module__}")
        
        clscontents = get_cls_content_list(cls, cls2defmethods, skipwhitespace=False)
        clscontentvals = [v[0] for v in clscontents]
        if "norefactors" in oprs_set:
            print("  norefactors working.")
            vallist, succ = ClassNoRefactor().refactor(clscontentvals, cls)
            norefactors.append( (cls, vallist, succ) )
        if "methodshuffles" in oprs_set:
            print("  methodshuffles working.")
            vallist, succ = ClassMethodShuffle().refactor(clscontentvals, cls)
            methodshuffles.append( (cls, vallist, succ) )
        if "appendinhmethodss" in oprs_set:
            print("  appendinhmethodss working.")
            vallist, succ = ClassAppendInheritedMethods().refactor(clscontentvals, cls, inheritance_info_eligible_classes_dict)
            appendinhmethodss.append( (cls, vallist, succ) )
        if "appendashuffle" in oprs_set:
            print("  appendashuffle working.")
            vallist, succ = ClassAppendInheritedMethods().refactor(clscontentvals, cls, inheritance_info_eligible_classes_dict, cls2outmethods, defmethods2cls)
            if succ:
                vallist, _ = ClassMethodShuffle().refactor(vallist, cls)
            appendashuffle.append( (cls, vallist, succ) )
            
        if "funcifs" in oprs_set:
            print("  funcifs working.")
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(clscontentvals, cls, func_refacotr.GroupRefactor(func_refacotr.IfBranchFliper()) )
            funcifs.append( (cls, vallist, succ) )
        if "funcloops" in oprs_set:
            print("  funcloops working.")
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(clscontentvals, cls, func_refacotr.GroupRefactor(func_refacotr.While2For(), func_refacotr.For2While()) )
            funcloops.append( (cls, vallist, succ) )
        if "funcparams" in oprs_set:
            print("  funcparams working.")
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(clscontentvals, cls, func_refacotr.GroupRefactor(func_refacotr.FnVarargAppender(), func_refacotr.FnKwargAppender()) )
            funcparams.append( (cls, vallist, succ) )
        if "funcvarrenamers" in oprs_set:
            print("  funcvarrenamers working.")
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(clscontentvals, cls, func_refacotr.GroupRefactor(func_refacotr.VarRenamer()) )
            funcvarrenamers.append( (cls, vallist, succ) )
        if "funcfndecorators" in oprs_set:
            print("  funcfndecorators working.")
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(clscontentvals, cls, func_refacotr.GroupRefactor(func_refacotr.FnDecorator('@timeing', '@measure_memory_usage')) )
            funcfndecorators.append( (cls, vallist, succ) )
        if "funccamelsnakeexchange" in oprs_set:
            print("  funccamelsnakeexchange working.")
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(clscontentvals, cls, func_refacotr.GroupRefactor(func_refacotr.CamelSnakeExchange()) )
            funccamelsnakeexchange.append( (cls, vallist, succ) )
        if "funclist2range" in oprs_set:
            print("  funclist2range working.")
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(clscontentvals, cls, func_refacotr.GroupRefactor(func_refacotr.List2Range()) )
            funclist2range.append( (cls, vallist, succ) )
        if "funccommlaw" in oprs_set:
            print("  funccommlaw working.")
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(clscontentvals, cls, func_refacotr.GroupRefactor(func_refacotr.CommLaw()) )
            funccommlaw.append( (cls, vallist, succ) )
        
        if "allw" in oprs_set:
            print("  allw working.")
            vallist, succs = clscontentvals, []
            vallist, succ = ClassAppendInheritedMethods().refactor(vallist, cls, inheritance_info_eligible_classes_dict, cls2outmethods, defmethods2cls)
            succs.append(succ)
            vallist, succ = ClassMethodShuffle().refactor(vallist, cls)
            succs.append(succ)
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.IfBranchFliper()) )
            succs.append(succ)
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.While2For(), func_refacotr.For2While()) )
            succs.append(succ)
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.FnVarargAppender(), func_refacotr.FnKwargAppender()) )
            succs.append(succ)
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.VarRenamer()) )
            succs.append(succ)
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.FnDecorator('@timeing', '@measure_memory_usage')) )
            succs.append(succ)
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.CamelSnakeExchange()) )
            succs.append(succ)
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.List2Range()) )
            succs.append(succ)
            vallist, succ = ClassApplyMethodOprWholeClass().refactor(vallist, cls, func_refacotr.GroupRefactor(func_refacotr.CommLaw()) )
            succs.append(succ)
            allw.append( (cls, vallist, any(succs)) )
        
        print(f"Done {cls.__qualname__} @ {cls.__module__}")

    for records_name in oprs:
        print(f"Dumping {records_name}...")
        records_val = eval(records_name)
        if not os.path.exists(f"data_{timetag}/class_{objlib.__name__}_{records_name}"):
            os.makedirs(f"data_{timetag}/class_{objlib.__name__}_{records_name}")
        with open(f"data_{timetag}/class_{objlib.__name__}_{records_name}/question.jsonl", "w") as fq, open(f"data_{timetag}/class_{objlib.__name__}_{records_name}/answer.jsonl", "w") as fa:
            fq.writelines([ f"{dumpq(*x, do_norm=do_norm)}\n" for x in records_val if x[1] is not None])

In [ ]:
import pandas, numpy, sklearn
timetag = datetime.datetime.now().strftime("%m%d%H%M%S")

work(pandas, do_check=True, timetag=timetag, oprs=['allw'])
print()
work(numpy, do_check=True, timetag=timetag, oprs=['allw'])
print()
work(sklearn, do_check=True, timetag=timetag, oprs=['allw'])
print()